In [1]:
import cv2 as cv
import numpy as np
import dlib
from imutils import face_utils
from pygame import mixer

# Initialize pygame mixer
mixer.init()
# Load the sound files
sound_drowsy = mixer.Sound('alert.mp3')  # Sound for drowsy
sound_sleepy = mixer.Sound('wake_up.mp3')  # Sound for sleepy

# Initialize face detector and predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Initialize variables
sleep = 0
drowsy = 0
active = 0
status = ""
color = (0, 0, 0)

# Function to calculate distance between two points
def dist(a, b):
    dst = np.linalg.norm(a-b)
    return dst

# Function to check if eyes are blinked
def blinked(a, b, c, d, e, f):
    up = dist(b, d) + dist(c, e)
    down = dist(a, f)
    ratio = up / (2.0 * down)

    if ratio > 0.25:
        return 2  # Blinked
    elif 0.25 >= ratio > 0.21:
        return 1  # Drowsy
    else:
        return 0  # Not blinked

# Function to calculate lip distance to detect yawning
def lip_distance(shape):
    top_lip = shape[50:53]
    top_lip = np.concatenate((top_lip, shape[61:64]))  # Top lip points
    low_lip = shape[56:59]
    low_lip = np.concatenate((low_lip, shape[65:68]))  # Bottom lip points

    top_mean = np.mean(top_lip, axis=0)
    low_mean = np.mean(low_lip, axis=0)

    distance = abs(top_mean[1] - low_mean[1])  # Calculate vertical distance between lips
    return distance

# Open webcam
cap = cv.VideoCapture(0)

while True:
    _, frame = cap.read()
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    faces = detector(gray)
    # face_frame = frame.copy()

    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()

        face_frame = frame.copy()
        cv.rectangle(face_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # Get facial landmarks
        landmarks = predictor(gray, face)
        landmarks = face_utils.shape_to_np(landmarks)

        # Check if eyes are blinked
        left_blink = blinked(landmarks[36], landmarks[37], landmarks[38], landmarks[41], landmarks[40], landmarks[39])
        right_blink = blinked(landmarks[42], landmarks[43], landmarks[44], landmarks[47], landmarks[46], landmarks[45])

        # Check for yawn (lip distance)
        distance = lip_distance(landmarks)

        # Update status based on blink detection and yawning
        if left_blink == 0 and right_blink == 0:
            sleep += 1
            drowsy = 0
            active = 0
            if sleep > 6:
                status = "SLEEPING!"
                color = (255, 0, 0)
                # Play sleep sound
                sound_sleepy.play()
        elif left_blink == 1 and right_blink == 1:
            sleep = 0
            active = 0
            drowsy += 1
            if drowsy > 6:
                status = "DROWSY!"
                color = (0, 0, 255)
                # Play drowsy sound
                sound_drowsy.play()
        elif distance > 35:  # If the mouth opens significantly, indicating a yawn
            status = "DROWSY! (Yawning)"
            color = (0, 0, 255)
            drowsy += 1
            if drowsy > 6:
                # Play drowsy sound for yawning
                sound_drowsy.play()
        else:
            drowsy = 0
            sleep = 0
            active += 1
            if active > 6:
                status = "ACTIVE!"
                color = (0, 255, 0)

        # Display the status on the frame
        cv.putText(frame, status, (100, 100), cv.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)

        # Draw landmarks
        for n in range(0, 68):
            (x, y) = landmarks[n]
            cv.circle(face_frame, (x, y), 1, (255, 255, 255), -1)

    # Show the video feed with landmarks
    cv.imshow("Frame", frame)
    cv.imshow("Result of detector", face_frame)

    # Exit condition (Press 'ESC' to stop)
    key = cv.waitKey(1)
    if key == 27:  # ESC key
        break

# Release the video capture and close OpenCV windows
cap.release()  # Stop the webcam
cv.destroyAllWindows()  # Close all OpenCV windows


ModuleNotFoundError: No module named 'cv2'